# Initialize Spark Session and Read in Dataset

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Load in the Postgress Driver
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
# Import Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
## CHANGE TO OUR S3 BUCKET:  url = "https://<bucket name>.s3.amazonaws.com/employee.csv"
spark.sparkContext.addFile(url)

## CHANGE CSV NAME:  NYSE_df = spark.read.option('header', 'true').csv(SparkFiles.get("NYSE_Data.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")


# Data Preprocessing

In [ ]:
NYSE_df.show(10)
print(NYSE_df.count())

In [ ]:
NYSE_df = NYSE_df.dropna()
print(NYSE_df.count())

In [ ]:
# Divide Inflation and Deflator by 100?

# GDP * 1M

# Multiply Population by 1M

# Drop unecessary columns

# Calculate Real GDP per Capita from GDP and Population


NYSE_df.reset_index(inplace=True, drop=True)
NYSE_df.head()

# Split Data into Training and Testing

In [ ]:
# Create our features
X = NYSE_df.drop('NYA', axis=1)

# Create our target
y = NYSE_df['NYA']

X.head()

In [ ]:
# Check features
X.describe()

In [ ]:
# Check the balance of our target values
y.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)

In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = brfc.predict(X_test)

balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

# Retraining the Model (if don't achieve desired accuracy score)